In [1]:
import numpy as np
import pandas as pd
#import shap
import joblib
#import time
from pathlib import Path
#import argparse
#import shap
import gc

#from sklearn.model_selection import train_test_split
#from sklearn.model_selection import StratifiedShuffleSplit
#from sklearn.preprocessing import RobustScaler,Normalizer,QuantileTransformer,PowerTransformer,StandardScaler
from sklearn.preprocessing import OrdinalEncoder
#,LabelEncoder,OneHotEncoder
from sklearn.metrics import mean_absolute_error
#,r2_score,mean_squared_log_error,
#from sklearn.metrics import make_scorer
#from sklearn.model_selection import cross_val_score,cross_validate

#import xgboost as xgb
import lightgbm as lgb
#from catboost import CatBoostRegressor
#from fastai.tabular.all import *
#import optuna

import warnings
warnings.filterwarnings(action="ignore")

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
np.set_printoptions(threshold=1000)
pd.options.display.max_info_columns=1000

DATA_PATH = Path('../data')
BASE_DIR = Path('../data')

In [2]:

# split data
col_rosters = [
    'playerId','date','teamId','status']
col_awards = ['playerId','date','awardPlayerTeamId','awardName']
col_twP = ['playerId','date','numberOfFollowers']
col_twT = ['teamId','date','numberOfFollowersT']
col_stand = [
    'date','teamId','divisionId','divisionRank','leagueRank',
    'leagueGamesBack','sportGamesBack','divisionGamesBack','wins',
    'losses','pct','runsAllowed','runsScored','divisionChamp',
    'divisionLeader','eliminationNumber','homeWins','homeLosses',
    'awayWins','awayLosses','lastTenWins','lastTenLosses',
    'extraInningWins','extraInningLosses','oneRunWins',
    'oneRunLosses','dayWins','dayLosses','nightWins',
    'nightLosses','grassWins','grassLosses','turfWins',
    'turfLosses','divWins','divLosses'
]
col_tran = ['playerId','date','typeCode']
col_scoreP = [
    'playerId','date','gamePk','home','teamId_score',
    'jerseyNum','battingOrder','gamesPlayedBatting','flyOuts',
    'groundOuts','runsScored','doubles','triples','homeRuns',
    'strikeOuts','baseOnBalls','intentionalWalks','hits','hitByPitch',
    'atBats','caughtStealing','stolenBases','groundIntoDoublePlay',
    'groundIntoTriplePlay','plateAppearances','totalBases','rbi',
    'leftOnBase','sacBunts','sacFlies','catchersInterference',
    'pickoffs','gamesPlayedPitching','gamesStartedPitching',
    'completeGamesPitching','shutoutsPitching','winsPitching',
    'lossesPitching','flyOutsPitching','airOutsPitching',
    'groundOutsPitching','runsPitching','doublesPitching',
    'triplesPitching','homeRunsPitching','strikeOutsPitching',
    'baseOnBallsPitching','intentionalWalksPitching','hitsPitching',
    'hitByPitchPitching','atBatsPitching','caughtStealingPitching',
    'stolenBasesPitching','inningsPitched','saveOpportunities',
    'earnedRuns','battersFaced','outsPitching','pitchesThrown',
    'balls','strikes','hitBatsmen','balks','wildPitches',
    'pickoffsPitching','rbiPitching','gamesFinishedPitching',
    'inheritedRunners','inheritedRunnersScored','catchersInterferencePitching',
    'sacBuntsPitching','sacFliesPitching','saves','holds','blownSaves',
    'assists','putOuts','errors','chances'
]
col_game = [
    'date','gamePk','detailedGameState','isTie','gameNumber',
    'doubleHeader','dayNight','scheduledInnings','gamesInSeries',
    'seriesDescription','homeId','homeWins','homeLosses','homeWinPct',
    'homeWinner','awayId','awayWins','awayLosses','awayWinPct',
    'awayWinner'
]

col_label = ['target1','target2','target3','target4']
col_sub = ['date','date_playerId','target1','target2','target3','target4']



In [3]:
# decode, clean, merge, clean
# 
def decodeFunc(json_str,date)->pd.DataFrame:
    if json_str!=json_str:
        out = pd.DataFrame()
    else: 
        out = pd.read_json(json_str)
        out['date'] = date
    return out

def splitdata(df,col_rosters,col_awards,col_twP,col_twT,col_stand,col_tran,col_scoreP,col_game):
    # in season
    target2018 = df[(df.date>=20180329)&(df.date<=20181028)]
    target2019 = df[(df.date>=20190320)&(df.date<=20191030)]
    target2020 = df[(df.date>=20200723)&(df.date<=20201028)]
    target2021 = df[(df.date>=20210228)&(df.date<=20211031)]
    df = pd.concat([target2018,target2019,target2020,target2021],axis=0)
    
    Label = pd.DataFrame()
    Games = pd.DataFrame()
    Rosters = pd.DataFrame()
    Scores = pd.DataFrame()
    Trans = pd.DataFrame()
    Awards = pd.DataFrame()
    TwP = pd.DataFrame()
    TwT = pd.DataFrame()
    Stand = pd.DataFrame() 
    #
    for index, test_df in df.iterrows():
        
        date = test_df['date']
        #====== label
        label = decodeFunc(test_df['nextDayPlayerEngagement'],date)
        label['engagementMetricsDate'] = pd.to_datetime(label['engagementMetricsDate'],format='%Y-%m-%d')
        label['engagementMetricsDate'] = label['engagementMetricsDate'].dt.strftime('%Y%m%d')
        label['date_playerId'] = label['engagementMetricsDate']+'_'+label['playerId'].astype(str)
        label = label.drop(columns=['engagementMetricsDate'])
        Label = pd.concat([Label,label],axis=0)
        sub_df = label[['date_playerId','date','playerId']]
        
        #====== Games
        games = decodeFunc(test_df['games'],date)
        if games.empty:
            games['gamePk'] = 0
            games['date'] = sub_df['date']
            col_game2 = [x for x in col_game if x not in ['gamePk','date']]
            for col in col_game2:
                games[col] = np.nan
        else:
            games = games[col_game]
            games = games[games.detailedGameState!='Postponed']
            games = games[~games['gamePk'].duplicated(keep='last')]
        Games = pd.concat([Games,games],axis=0)
    
        #====== Rosters
        rosters = decodeFunc(test_df['rosters'],date)
        if rosters.empty:
            rosters['playerId'] = sub_df['playerId']
            rosters['date'] = sub_df['date']
            rosters['date_playerId'] = sub_df['date_playerId']
            rosters['teamId'] = 0
            rosters['status'] = 'unknown'
        else:
            rosters = rosters[col_rosters]
            rosters['teamId'].fillna(0,inplace=True)
            rosters['status'].fillna('unknown',inplace=True)
            rosters['date_playerId'] = rosters['date'].astype(str)+'_'+rosters['playerId'].astype(str)
            rosters = rosters[~rosters['date_playerId'].duplicated(keep='last')]
        Rosters = pd.concat([Rosters,rosters],axis=0)
        
        #====== Player Scores
        scores = decodeFunc(test_df['playerBoxScores'],date)
        if scores.empty:
            scores['playerId'] = sub_df['playerId']
            scores['date'] = sub_df['date']
            col_score2 = [x for x in col_scoreP if x not in ['playerId','date']]
            for col in col_score2:
                scores[col] = np.nan
            scores['date_playerId'] = sub_df['date_playerId']
        else:
            scores.rename(columns={'teamId':'teamId_score'},inplace=True)
            scores = scores[col_scoreP]
            scores['date_playerId'] = scores['date'].astype(str)+'_'+scores['playerId'].astype(str)
            scores = scores[~scores['date_playerId'].duplicated(keep='last')]
        Scores = pd.concat([Scores,scores],axis=0)
        
        #====== Transaction
        trans = decodeFunc(test_df['transactions'],date) 
        if trans.empty:
            trans['playerId'] = sub_df['playerId']
            trans['date'] = sub_df['date']
            trans['bTrans'] = 0
            trans['date_playerId'] = sub_df['date_playerId']
        else:
            trans = trans[col_tran]
            trans['bTrans'] = trans['typeCode'].apply(lambda x: 1 if x==x else 0)
            trans['date_playerId'] = trans['date'].astype(str)+'_'+trans['playerId'].astype(str)
            trans = trans[~trans['date_playerId'].duplicated(keep='last')]
            #trans['typeCode'].fillna('unknown',inplace=True)
        Trans = pd.concat([Trans,trans],axis=0)
        
        #====== Awards
        awards = decodeFunc(test_df['awards'],date)
        if awards.empty:
            awards['playerId'] = sub_df['playerId']
            awards['date'] = sub_df['date']
            awards['awardPlayerTeamId'] = 0
            awards['awardName'] = 'noaward'
            awards['bAward'] = 0
            awards['date_playerId'] = sub_df['date_playerId']
        else:
            awards = awards[col_awards]
            #awards = awards.groupby(['playerId','date','awardPlayerTeamId'])['awardName'].sum('_')
            #awards = awards.reset_index()
            awards['awardPlayerTeamId'].fillna(0,inplace=True)
            awards['awardName'].fillna('noward',inplace=True)
            awards['bAward'] = awards['awardName'].apply(lambda x: 1 if x==x else 0)
            awards['date_playerId'] = awards['date'].astype(str)+'_'+awards['playerId'].astype(str)
            awards = awards[~awards['date_playerId'].duplicated(keep='last')]
        Awards = pd.concat([Awards,awards],axis=0)

        #====== Twitter player
        twitP = decodeFunc(test_df['playerTwitterFollowers'],date)
        if twitP.empty:
            twitP['playerId'] = sub_df['playerId']
            twitP['date'] = sub_df['date']
            twitP['numberOfFollowers'] = np.nan
            twitP['date_playerId'] = sub_df['date_playerId']
        else:
            twitP = twitP[col_twP]
            twitP['date_playerId'] = twitP['date'].astype(str)+'_'+twitP['playerId'].astype(str)
            twitP = twitP[~twitP['date_playerId'].duplicated(keep='last')]
        TwP = pd.concat([TwP,twitP],axis=0)

        #====== Team standing
        stand = decodeFunc(test_df['standings'],date)
        if stand.empty:
            stand['teamId'] = 0
            stand['date'] = sub_df['date']
            col_stand2 = [x for x in col_stand if x not in ['teamId','date']]
            for col in col_stand2:
                stand[col] = np.nan
            stand.rename(columns=lambda x: x+'T', inplace=True)
        else:
            stand = stand[col_stand]
            stand.rename(columns=lambda x: x+'T', inplace=True)
        Stand = pd.concat([Stand,stand],axis=0)

        #====== Twitter Team
        twitT = decodeFunc(test_df['teamTwitterFollowers'],date)
        if twitT.empty:
            twitT['teamId'] = 0
            twitT['date'] = sub_df['date']
            twitT['numberOfFollowersT'] = np.nan
        else:
            twitT.rename(columns={'numberOfFollowers':'numberOfFollowersT'},inplace=True)
            twitT = twitT[col_twT]
        TwT = pd.concat([TwT,twitT],axis=0)
    
    return Label,Games,Rosters,Awards,TwP,Stand,TwT,Trans,Scores



In [4]:

def merge(targets,players,rosters,awards,twP,stand,twT,trans,scoreP)->pd.DataFrame():
    
    targets2 = targets.merge(players,how='left',on=['playerId'])
    targets2 = targets2[targets2.age.notnull()]
    
    targets3 = targets2.merge(rosters[['teamId','status','date_playerId']],how='left',on=['date_playerId',])
    targets3['teamId'].fillna(0,inplace=True)
    targets3['status'].fillna('unknown',inplace=True)
    
    targets4 = targets3.merge(awards[['bAward','date_playerId']],how='left',on=['date_playerId'])
    
    targets5 = targets4.merge(twP[['date_playerId','numberOfFollowers']],how='left',on=['date_playerId'])
    targets5 = targets5.sort_values(by='date')
    targets5['numberOfFollowers'] = targets5.groupby('playerId')['numberOfFollowers'].fillna(method='bfill')
    targets5['numberOfFollowers'] = targets5.groupby('playerId')['numberOfFollowers'].fillna(method='ffill')
    targets5['numberOfFollowers'].fillna(0,inplace=True)
    
    # team stand
    targets6 = targets5.merge(stand,how='left',left_on=['teamId','date'],right_on=['teamIdT','dateT'])
    #targets6 = targets6.fillna(0)
    targets6[
        ['leagueGamesBackT','sportGamesBackT','divisionGamesBackT','eliminationNumberT']
    ] = targets6[
        ['leagueGamesBackT','sportGamesBackT','divisionGamesBackT','eliminationNumberT']
    ].replace('-',0)
    targets6.drop(columns=['teamIdT','dateT'],inplace=True)
    
    targets7 = targets6.merge(twT,how='left',on=['teamId','date'])
    targets7 = targets7.sort_values(by='date')
    targets7['numberOfFollowersT'] = targets7.groupby('teamId')['numberOfFollowersT'].fillna(method='bfill')
    targets7['numberOfFollowersT'] = targets7.groupby('teamId')['numberOfFollowersT'].fillna(method='ffill')
    targets7['numberOfFollowersT'].fillna(0,inplace=True)
    
    targets8 = targets7.merge(trans[['date_playerId','bTrans']],how='left',on=['date_playerId'])
    
    targets9 = targets8.merge(scoreP.drop(columns=['playerId','date']),how='left',on=['date_playerId'])
    
    #targets9['teamId'] = targets9[['teamId','teamId_score']].apply(lambda x:x['teamId_score'] if x['teamId_score']==x['teamId_score'] and x['teamId_score']!=x['teamId'] else x['teamId'],axis=1)
    targets9 = targets9.drop(columns=['teamId_score'])

    return targets9



def clean(targets, games,cg_key,cg_int,cg_cat,cg_flag,
          ct_key,ct_int,ct_cat,ct_flag,col_float,encoder1,encoder2)->pd.DataFrame():
    
    games.rename(columns={
        'homeWins':'homeWinsG','homeLosses':'homeLossesG','awayWins':'awayWinsG',
        'awayLosses':'awayLossesG'},inplace=True)
    
    #
    targets['eliminationNumberT'] = targets['eliminationNumberT'].apply(lambda x:0 if str(x)=='E' else x)
    targets['gamePk'] = targets['gamePk'].fillna(0)
    targets[ct_cat] = targets[ct_cat].fillna('unknown')
    targets[ct_int] = targets[ct_int].fillna(0)
    targets[ct_flag] = targets[ct_flag].fillna(0)
    
    targets[ct_int] = targets[ct_int].apply(pd.to_numeric)
    targets[ct_int] = targets[ct_int].fillna(0)
    targets[ct_cat] = targets[ct_cat].astype('str')
    targets[ct_int] = targets[ct_int].astype(int)
    targets[ct_flag] = targets[ct_flag].astype(int)
    
    #for col in ct_flag:
    #    targets[col] = targets[col].apply(lambda x:'True' if str(x)=='1' else x)
    #    targets[col] = targets[col].apply(lambda x:'False' if str(x)=='0' else x)
    #    targets[col] = targets[col].apply(lambda x:'True' if str(x)=='1.0' else x)
    #    targets[col] = targets[col].apply(lambda x:'False' if str(x)=='0.0' else x)

    targets = targets.sort_values(by='date')
    targets2 = targets.merge(games,how='left',on=['gamePk','date'])
    # correct columns
    #targets2['divisionId'] = targets2['divisionId'].astype(float).astype(int)
    
    #game columns
    targets2[cg_cat] = targets2[cg_cat].fillna('unknown')
    targets2[cg_int] = targets2[cg_int].fillna(0)
    targets2[cg_flag] = targets2[cg_flag].fillna(0)
    
    targets2[cg_cat] = targets2[cg_cat].astype('str')
    targets2[cg_int] = targets2[cg_int].astype(float).astype(int)
    targets2[cg_flag] = targets2[cg_flag].astype(int)
   
    #for col in cg_flag:
    #    targets2[col] = targets2[col].apply(lambda x:'True' if x=='1' else x)
    #    targets2[col] = targets2[col].apply(lambda x:'True' if x=='1.0' else x)
    #    targets2[col] = targets2[col].apply(lambda x:'False' if x=='0' else x)
    #    targets2[col] = targets2[col].apply(lambda x:'False' if x=='0.0' else x)
    
    targets2[col_float] = targets2[col_float].fillna(0.0)    
    targets2[col_float] = targets2[col_float].astype(float)
    
    #
    col_encode = ct_cat+cg_cat
    targets2[col_encode] = encoder1.fit_transform(targets2[col_encode])
    #
    df_game = targets2[targets2['gamePk']!=0]
    df_game.drop(columns=['gamePk','teamId','homeId','awayId'],inplace=True)
    
    df_nogame = targets2[targets2['gamePk']==0]
    
    col_dropnogame = [
        'gamePk','scheduledInnings','gameNumber','gamesInSeries','homeWinsG',
        'homeLossesG','awayWinsG','awayLossesG','homeId','awayId',
        'detailedGameState','doubleHeader','dayNight','seriesDescription',
        'isTie','homeWinner','awayWinner','homeWinPct','awayWinPct'
    ] 
    df_nogame.drop(columns=col_dropnogame,inplace=True)
    
    return df_game, df_nogame, encoder1

In [5]:
def add_stat(df_game,df_nogame,labelStat):
    # dateDate, year, month
    labelStat.rename(columns={'playerId':'playerIdKey'},inplace=True)
    df_game['dateDate'] = pd.to_datetime(df_game['date'],format='%Y%m%d')
    df_game['month'] = df_game['dateDate'].dt.month
    df_game['playerIdKey'] = df_game['date_playerId'].map(lambda x: int(x.split('_')[1]))
    
    df_nogame['dateDate'] = pd.to_datetime(df_nogame['date'],format='%Y%m%d')
    df_nogame['month'] = df_nogame['dateDate'].dt.month
    df_nogame['playerIdKey'] = df_nogame['date_playerId'].map(lambda x: int(x.split('_')[1]))
    #label['day'] = label['date'].dt.day
    
    df_game = df_game.merge(labelStat,how='left',on=['playerIdKey','month'])
    df_nogame = df_nogame.merge(labelStat,how='left',on=['playerIdKey','month'])
    
    df_game.fillna(0.0,inplace=True)
    df_nogame.fillna(0.0,inplace=True)
    
    df_game = df_game.drop(columns=['dateDate','month','playerIdKey'])
    df_nogame = df_nogame.drop(columns=['dateDate','month','playerIdKey'])
    
    return df_game, df_nogame



def add_lag(df_game,df_nogame,label,LAG=1,WIN_SIZE=[3]):
    col_label = ['target1','target2','target3','target4']
    col_mean = ['target1Mean','target2Mean','target3Mean','target4Mean']
    col_max = ['target1Max','target2Max','target3Max','target4Max']
    col_min = ['target1Min','target2Min','target3Min','target4Min']
    col_std = ['target1Std','target2Std','target3Std','target4Std']
    col_median = ['target1Med','target2Med','target3Med','target4Med']
    
    label = label.sort_values(by='date').reset_index(drop=True).copy(deep=True)
    col_lag = ['date_playerId']
    
    # lag targets
    for i in range(1,LAG+1):
        col_new = [x+'_Lag'+str(i) for x in col_label]
        label[col_new] = label.groupby('playerId')[col_label].shift(i)
        col_lag = col_lag+col_new
    
    col_roll = col_lag
    #roll() mean, max, min, std, median: run every iter, 4*5*Nsize
    for i in WIN_SIZE:
        col_meanR = [x+'_Size'+str(i) for x in col_mean]
        label[col_meanR] = label.groupby('playerId')[col_label].rolling(window=i).mean().reset_index(0,drop=True)

        col_maxR = [x+'_Size'+str(i) for x in col_max]
        label[col_maxR] = label.groupby('playerId')[col_label].rolling(window=i).max().reset_index(0,drop=True)

        col_minR = [x+'_Size'+str(i) for x in col_min]
        label[col_minR] = label.groupby('playerId')[col_label].rolling(window=i).min().reset_index(0,drop=True)

        col_medianR = [x+'_Size'+str(i) for x in col_median]
        label[col_medianR] = label.groupby('playerId')[col_label].rolling(window=i).median().reset_index(0,drop=True)

        col_stdR = [x+'_Size'+str(i) for x in col_std]
        label[col_stdR] = label.groupby('playerId')[col_label].rolling(window=i).std().reset_index(0,drop=True)
        col_roll = col_roll+col_meanR+col_maxR+col_minR+col_medianR+col_stdR
    
    label = label[col_roll].dropna()
    df_game = df_game.merge(label,how='left',on=['date_playerId'])
    df_nogame = df_nogame.merge(label,how='left',on=['date_playerId'])
    
    df_game.dropna(inplace=True)
    df_nogame.dropna(inplace=True)
    
    return df_game, df_nogame,label.columns.tolist()[1:]

In [8]:
#df_in = pd.read_csv(BASE_DIR/'train_updated.csv')
#players = pd.read_csv(DATA_PATH/'player.csv')
#len(df_in.columns),len(df_in)
#df_in.head(2)

In [9]:
'''
#
label,games,rosters,awards,twitP,stand,twitT,trans,scores = splitdata(
    df_in,col_rosters,col_awards,col_twP,col_twT,col_stand,col_tran,col_scoreP,col_game
)

del df_in
gc.collect()
'''

'\n#\nlabel,games,rosters,awards,twitP,stand,twitT,trans,scores = splitdata(\n    df_in,col_rosters,col_awards,col_twP,col_twT,col_stand,col_tran,col_scoreP,col_game\n)\n\ndel df_in\ngc.collect()\n'

In [10]:
'''#
targets = merge(label,players,rosters,awards,twitP,stand,twitT,trans,scores)

del players
del rosters
del awards
del twitP
del stand
del twitT
del trans
del scores
gc.collect()'''

'#\ntargets = merge(label,players,rosters,awards,twitP,stand,twitT,trans,scores)\n\ndel players\ndel rosters\ndel awards\ndel twitP\ndel stand\ndel twitT\ndel trans\ndel scores\ngc.collect()'

In [11]:
#for col in cg_flag:
#    display(games[col].dtype)
    #display(targets[col].dtype)

In [12]:
# clean data: key, int, cat, flag for tragets and games

ct_key = [
    'date_playerId','teamId','date','gamePk',
]
ct_int = [
    'age','ageStart','yearPlay','numberOfFollowers',
    # stand
    'divisionRankT','leagueRankT','leagueGamesBackT','sportGamesBackT',
    'divisionGamesBackT','winsT','lossesT','runsAllowedT','runsScoredT',
    'eliminationNumberT','homeWinsT','homeLossesT','awayWinsT','awayLossesT',
    'lastTenWinsT','lastTenLossesT','extraInningWinsT','extraInningLossesT',
    'oneRunWinsT','oneRunLossesT','dayWinsT','dayLossesT','nightWinsT',
    'nightLossesT','grassWinsT','grassLossesT','turfWinsT','turfLossesT','divWinsT','divLossesT',
    #
    'numberOfFollowersT',
    # score
    'jerseyNum','battingOrder','flyOuts','groundOuts','runsScored','doubles','triples',
    'homeRuns','strikeOuts','baseOnBalls','intentionalWalks','hits','hitByPitch',
    'atBats','caughtStealing','stolenBases','groundIntoDoublePlay','groundIntoTriplePlay',
    'plateAppearances','totalBases','rbi','leftOnBase','sacBunts','sacFlies',
    'catchersInterference','pickoffs',
    'flyOutsPitching','airOutsPitching','groundOutsPitching','runsPitching','doublesPitching',
    'triplesPitching','homeRunsPitching','strikeOutsPitching','baseOnBallsPitching',
    'intentionalWalksPitching','hitsPitching','hitByPitchPitching','atBatsPitching',
    'caughtStealingPitching','stolenBasesPitching','inningsPitched','earnedRuns',
    'battersFaced','outsPitching','pitchesThrown','balls','strikes','hitBatsmen',
    'balks','wildPitches','pickoffsPitching','rbiPitching','gamesFinishedPitching',
    'inheritedRunners','inheritedRunnersScored','catchersInterferencePitching',
    'sacBuntsPitching','sacFliesPitching','assists','putOuts','errors','chances',
    #
    
    
]
ct_cat = [
    'playerId','birthCity','birthStateProvince','birthCountry',
    'primaryPositionName','status',
    # stand
    'divisionIdT','divisionChampT','divisionLeaderT',
    
]
ct_flag = [
    'bAward','bTrans',
    # score        
    'home','gamesPlayedBatting','gamesPlayedPitching','gamesStartedPitching',
    'completeGamesPitching','shutoutsPitching','winsPitching','lossesPitching',
    'saveOpportunities','saves','holds','blownSaves',
]

## game
cg_key = ['gamePk','date']

cg_int = ['scheduledInnings','gameNumber','gamesInSeries','homeWinsG','homeLossesG',
           'awayWinsG','awayLossesG','homeId','awayId']

cg_cat = ['detailedGameState','doubleHeader','dayNight',
           'seriesDescription']

cg_flag = ['isTie','homeWinner','awayWinner'] 

col_float = [
    'homeWinPct','awayWinPct',
    'target1','target2','target3','target4',
    'BMI','pctT'
]

#col_float = [x for x in games.columns.tolist()+targets.columns.tolist() 
#             if x not in cg_key+cg_int+cg_cat+cg_flag+ct_key+ct_int+ct_cat+ct_flag+col_label]

In [13]:
#for col in ct_int:
#    print(col)
#    if col=='eliminationNumberT':
#        continue
#    targets[col] = pd.to_numeric(targets[col])
#    targets[col].astype(int)
#targets.jerseyNum.unique()

In [14]:
cat_encoder = OrdinalEncoder()
cat_encoder2 = OrdinalEncoder()

#
df_game, df_nogame, encoder = clean(targets, games,cg_key,cg_int,cg_cat,cg_flag,
                           ct_key,ct_int,ct_cat,ct_flag,col_float,cat_encoder,cat_encoder2)

del targets
del games
gc.collect()

NameError: name 'targets' is not defined

In [12]:
#df_game.to_csv('df_g1.csv',index=False)
#df_nogame.to_csv('df_n1.csv',index=False)

df_game= pd.read_csv('df_g1.csv')
df_nogame = pd.read_csv('df_n1.csv')

In [13]:
labelStat = pd.read_csv(DATA_PATH / 'labelStat.csv')
labelAll = pd.read_csv(DATA_PATH / 'targetsAll.csv')

col_stat = labelStat.columns.tolist()[2:]


In [8]:
#labelStat.head(1)

In [10]:
labelAll.tail(1)

,playerId,target1,target2,target3,target4,date,date_playerId
1395296,542208,0.0,0.00359,0.0,0.085521,20210718,20210719_542208


In [49]:
df_game2, df_nogame2 = add_stat(df_game, df_nogame,labelStat)
#df_game2, df_nogame2,col_lag = add_lag(df_game1, df_nogame1,labelAll,10,[6,3,15])
lag_name = 'stat'
SAVE=True


In [ ]:
df_game2.tail(5)
#df_nogame.head(1)

In [24]:
#np.save('encoder_all.npy', encoder.categories_)

In [ ]:
df_game1.tail(2)
#df_nogame.tail(2)

In [18]:
col_key = ['date','date_playerId']

In [19]:
# target1: player+team
col_t1_game = [
    'rbi','homeRuns','numberOfFollowers','numberOfFollowersT',
    'plateAppearances','totalBases','ageStart','primaryPositionName',
    'dayNight','pctT','inningsPitched','grassWinsT','grassLossesT','dayWinsT','age',
    'atBatsPitching','hitsPitching','gamesStartedPitching','winsPitching','leagueRankT',
    'strikeOutsPitching','divWinsT','playerId','hits','awayLossesG',
    'divisionGamesBackT','BMI','saves','homeLossesT','awayWinsG',
    'lastTenWinsT','leagueGamesBackT','yearPlay','runsPitching',
    'gamesInSeries','oneRunWinsT','airOutsPitching','winsT',
    'birthCity','divisionRankT','bAward','earnedRuns',
    'divLossesT','awayLossesT','extraInningLossesT','homeWinsG',
    'awayWinPct','turfWinsT','pitchesThrown','atBats','divisionIdT',
    'completeGamesPitching','strikes','nightWinsT','outsPitching','extraInningWinsT', 
    'homeWinsT', 'awayWinsT','bTrans'
]


col_t1_nogame = [
    'status','numberOfFollowers','primaryPositionName','age',
    'ageStart','yearPlay','numberOfFollowersT','playerId',
    'divLossesT','runsAllowedT','BMI','dayWinsT','pctT',
    'birthCity', 'birthCountry', 'grassWinsT','bTrans'
    
]

In [20]:
# target2: 
col_t2_game = [
    'numberOfFollowers','plateAppearances','numberOfFollowersT','ageStart','dayNight',
    'age','atBats','totalBases','primaryPositionName','leagueRankT','pctT','saves','yearPlay',
    'BMI','rbi','playerId','homeRuns','homeWinsT','gamesStartedPitching','runsScored',
    'birthCountry','lastTenLossesT','runsScoredT','grassWinsT','birthStateProvince','gamesInSeries',
    'birthCity','hits','winsPitching','sportGamesBackT','dayLossesT','awayWinPct','strikeOutsPitching',
    'divisionIdT','lastTenWinsT','grassLossesT','homeLossesG','awayWinsG','lossesPitching',
    'leagueGamesBackT','outsPitching','saveOpportunities','inningsPitched',
    'divisionGamesBackT','awayLossesT','leftOnBase','baseOnBalls','lossesT','turfWinsT',
    'homeLossesT', 'bAward','bTrans'
]

col_t2_nogame = [
    'status','numberOfFollowers','numberOfFollowersT','yearPlay','runsAllowedT','ageStart',
    'divisionRankT','pctT','lossesT','divLossesT','divWinsT','playerId','leagueGamesBackT','leagueRankT',
    'winsT','nightLossesT','dayLossesT','awayLossesT','bTrans'
]

In [21]:
# target3: player score
col_t3_game = [
    'numberOfFollowers','homeRuns','gamesStartedPitching','pitchesThrown','rbi','numberOfFollowersT',
    'ageStart','totalBases','inningsPitched','runsPitching','strikes','leagueRankT','playerId',
    'strikeOutsPitching','battersFaced','age','lossesPitching','yearPlay','pctT','extraInningLossesT',
    'plateAppearances','saves','balls','primaryPositionName','outsPitching','winsPitching',
    'leagueGamesBackT','dayWinsT','awayWinsG','earnedRuns','rbiPitching','atBatsPitching','BMI','divisionIdT',
    'birthStateProvince','divisionGamesBackT','airOutsPitching','birthCity','bAward','dayNight',
    'winsT','sportGamesBackT','homeLossesT','turfWinsT','divisionRankT','grassWinsT','birthCountry',
    'runsAllowedT','atBats','runsScored','gamesInSeries','lastTenLossesT','hits','awayWinPct',
    'homeWinsT','lastTenWinsT','dayLossesT','lossesT','runsScoredT','leftOnBase',
    'saveOpportunities','awayLossesT','baseOnBalls','grassLossesT','bTrans'
]

col_t3_nogame = [
    'numberOfFollowers','status','ageStart','playerId','age','numberOfFollowersT','yearPlay',
    'birthCity','birthStateProvince','BMI','pctT','runsAllowedT','primaryPositionName',
    'divLossesT','lossesT','divisionIdT','awayWinsT','birthCountry','leagueGamesBackT','bTrans'
]

In [22]:
# target4
col_t4_game = [
    'numberOfFollowers','ageStart','yearPlay','numberOfFollowersT','plateAppearances',
    'playerId','totalBases','homeWinsT','gamesStartedPitching','saves','pctT',
    'dayNight','BMI','age','birthCity','birthStateProvince','primaryPositionName','runsScoredT',
    'homeLossesT','birthCountry','leagueRankT','awayLossesT','winsPitching','homeRuns',
    'awayWinPct','rbi','gamesInSeries','awayWinsT','sportGamesBackT','leagueGamesBackT',
    'runsAllowedT','grassWinsT','dayLossesT','lossesT','atBats','nightWinsT','divisionIdT',
    'awayWinsG','divisionGamesBackT','hits','oneRunLossesT','lastTenLossesT','divLossesT',
    'divisionRankT','dayWinsT','runsScored','extraInningLossesT','grassLossesT','divWinsT',
    'awayLossesG', 'turfLossesT','bTrans'
]

col_t4_nogame = ['numberOfFollowers','yearPlay','status','numberOfFollowersT','ageStart',
                 'age','playerId','primaryPositionName','birthStateProvince','divLossesT',
                 'birthCity','BMI','pctT','runsAllowedT','birthCountry','divWinsT',
                 'runsScoredT','winsT','lossesT','dayLossesT','nightLossesT','awayLossesT',
                 'divisionIdT','nightWinsT','extraInningLossesT','leagueGamesBackT','awayWinsT',
                 'leagueRankT','divisionGamesBackT','lastTenWinsT','grassLossesT','bTrans'
]

In [54]:
DATE_START = 20180101
SPLITDATE_pred = 20210630

DATE_END = 20210717

labelAll = labelAll[(labelAll.date>20210101)&(labelAll.date<=SPLITDATE_pred)]
labelAll['dateDate'] = labelAll['date_playerId'].map(lambda x: int(x.split('_')[0]))
labelAll['dateDate'] = pd.to_datetime(labelAll['dateDate'],format='%Y%m%d')
labelAll['dateDate'] = labelAll['dateDate'] + pd.Timedelta(days=1)
labelAll['date_playerId'] = labelAll['dateDate'].dt.strftime('%Y%m%d')+'_'+labelAll['playerId'].astype(str)
labelAll.drop(columns='dateDate',inplace=True)

'''final_col1 = col_key+col_label+col_t1_game+col_stat+col_lag
final_col2 = col_key+col_label+col_t2_game+col_stat+col_lag
final_col3 = col_key+col_label+col_t3_game+col_stat+col_lag
final_col4 = col_key+col_label+col_t4_game+col_stat+col_lag'''

final_col1 = col_key+col_label+col_t1_game+col_stat
final_col2 = col_key+col_label+col_t2_game+col_stat
final_col3 = col_key+col_label+col_t3_game+col_stat
final_col4 = col_key+col_label+col_t4_game+col_stat

final_col1n = col_key+col_label+col_t1_nogame+col_stat
final_col2n = col_key+col_label+col_t2_nogame+col_stat
final_col3n = col_key+col_label+col_t3_nogame+col_stat
final_col4n = col_key+col_label+col_t4_nogame+col_stat

dtr_g1 = df_game2[(df_game2.date<DATE_SPLIT)&(df_game2.date>=DATE_START)][final_col1]
dtt_g1 = df_game2[(df_game2.date>=DATE_SPLIT)&(df_game2.date<=DATE_END)][final_col1]
dtr_n1 = df_nogame2[(df_nogame2.date<DATE_SPLIT)&(df_nogame2.date>=DATE_START)][final_col1n]
dtt_n1 = df_nogame2[(df_nogame2.date>=DATE_SPLIT)&(df_nogame2.date<=DATE_END)][final_col1n]

dtr_g2 = df_game2[(df_game2.date<DATE_SPLIT)&(df_game2.date>=DATE_START)][final_col2]
dtt_g2 = df_game2[(df_game2.date>=DATE_SPLIT)&(df_game2.date<=DATE_END)][final_col2]
dtr_n2 = df_nogame2[(df_nogame2.date<DATE_SPLIT)&(df_nogame2.date>=DATE_START)][final_col2n]
dtt_n2 = df_nogame2[(df_nogame2.date>=DATE_SPLIT)&(df_nogame2.date<=DATE_END)][final_col2n]

#
dtr_g3 = df_game2[(df_game2.date<DATE_SPLIT)&(df_game2.date>=DATE_START)][final_col3]
dtt_g3 = df_game2[(df_game2.date>=DATE_SPLIT)&(df_game2.date<=DATE_END)][final_col3]
dtr_n3 = df_nogame2[(df_nogame2.date<DATE_SPLIT)&(df_nogame2.date>=DATE_START)][final_col3n]
dtt_n3 = df_nogame2[(df_nogame2.date>=DATE_SPLIT)&(df_nogame2.date<=DATE_END)][final_col3n]

#
dtr_g4 = df_game2[(df_game2.date<DATE_SPLIT)&(df_game2.date>=DATE_START)][final_col4]
dtt_g4 = df_game2[(df_game2.date>=DATE_SPLIT)&(df_game2.date<=DATE_END)][final_col4]
dtr_n4 = df_nogame2[(df_nogame2.date<DATE_SPLIT)&(df_nogame2.date>=DATE_START)][final_col4n]
dtt_n4 = df_nogame2[(df_nogame2.date>=DATE_SPLIT)&(df_nogame2.date<=DATE_END)][final_col4n]



In [35]:

#dtr_g2.to_csv('df_targ2_gtr.csv',index=False)
#dtt_g2.to_csv('df_targ2_gtt.csv',index=False)
#dtr_n2.to_csv('df_targ2_ntr.csv',index=False)
#dtt_n2.to_csv('df_targ2_ntt.csv',index=False)

In [55]:
params1 = {
    #'device_type':'gpu',
    #'gpu_device_id':1,
    #'gpu_platform_id': 0,
    'n_jobs' : 4,
    'objective': 'mae', 'learning_rate': 0.06400959575202655, 'n_estimators': 4492, 'max_depth': 11, 
    'num_leaves': 36693, 'max_bin': 166, 'feature_fraction': 0.6577763017247104, 'bagging_fraction': 0.9102311988250655, 
    'bagging_freq': 7, 'min_sum_hessian_in_leaf': 8, 'reg_alpha': 0.00022893773473705515, 
    'reg_lambda': 0.00048731841698770753,'num_iterations': 10000, 'early_stopping_round': 100
}

params2 = {
    #'device_type':'gpu',
    #'gpu_device_id':1,
    #'gpu_platform_id': 0,
    #'gpu_use_dp': False,
    'n_jobs' : 4,
    'seed':42,'objective': 'mae', 'metric': 'l1', 'feature_pre_filter': False, 
    'lambda_l1': 3.731605225849285, 'lambda_l2': 0.02803980626777797, 'num_leaves': 8, 
    'feature_fraction': 0.5, 'bagging_fraction': 0.5262728428461787, 'bagging_freq': 3, 
    'min_child_samples': 20, 'num_iterations': 10000, 'early_stopping_round': 100
}

'''
params2 = {
    'objective': 'huber', 'learning_rate': 0.04153405228128592, 'n_estimators': 1074, 'max_depth': 8, 
    # 'num_leaves': 21473, 'max_bin': 187, 'feature_fraction': 0.7866249424381657, 'bagging_fraction': 0.8835990951694892, 
    'bagging_freq': 93, 'min_sum_hessian_in_leaf': 7, 'reg_alpha': 0.099327837369119, 'reg_lambda': 1.737834771403329e-05,
    'num_iterations': 10000,'early_stopping_round': 100
}
'''
params3 = {
    #'device_type':'gpu',
    #'gpu_device_id':1,
    #'gpu_platform_id': 0,
    #'gpu_use_dp': False,
    'n_jobs' : 4,
    'seed':42,'objective': 'mae', 'metric': 'l1', 'feature_pre_filter': False, 
    'lambda_l1': 7.654830305013684, 'lambda_l2': 4.14748542765967e-07, 'num_leaves': 252, 
    'feature_fraction': 0.7200000000000001, 'bagging_fraction': 1.0, 'bagging_freq': 0, 
    'min_child_samples': 20, 'num_iterations': 10000, 'early_stopping_round': 100
}

params4 = {
    #'device_type':'gpu',
    #'gpu_device_id':1,
    #'gpu_platform_id': 0,
    #'gpu_use_dp': False,
    'n_jobs' : 4,
    'seed':42,'objective': 'mae', 'metric': 'l1', 'feature_pre_filter': False, 
    'lambda_l1': 9.486880706514734e-08, 'lambda_l2': 0.005143767850872896, 'num_leaves': 246, 
    'feature_fraction': 0.5479999999999999, 'bagging_fraction': 0.5238463354446826, 'bagging_freq': 5, 
    'min_child_samples': 20, 'num_iterations': 10000, 'early_stopping_round': 100
}

In [56]:
# game
model1 = lgb.LGBMRegressor(**params1)
model1.fit(dtr_g1.drop(columns=col_label+col_key).values,dtr_g1['target1'].values,
           eval_set=[(dtt_g1.drop(columns=col_label+col_key).values,dtt_g1['target1'].values)],verbose=100)

[LightGBM] [Warning] bagging_fraction is set=0.9102311988250655, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9102311988250655
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Warning] feature_fraction is set=0.6577763017247104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6577763017247104
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 2.49721
Early stopping, best iteration is:
[44]	valid_0's l1: 2.49228


LGBMRegressor(bagging_fraction=0.9102311988250655, bagging_freq=7,
              early_stopping_round=100, feature_fraction=0.6577763017247104,
              learning_rate=0.06400959575202655, max_bin=166, max_depth=11,
              min_sum_hessian_in_leaf=8, n_estimators=4492, n_jobs=4,
              num_iterations=10000, num_leaves=36693, objective='mae',
              reg_alpha=0.00022893773473705515,
              reg_lambda=0.00048731841698770753)

In [57]:
#
model2 = lgb.LGBMRegressor(**params2)
model2.fit(dtr_g2.drop(columns=col_label+col_key).values,dtr_g2['target2'].values,
           eval_set=[(dtt_g2.drop(columns=col_label+col_key).values,dtt_g2['target2'].values)],verbose=100)


Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 4.53435
Early stopping, best iteration is:
[7]	valid_0's l1: 3.61827


LGBMRegressor(bagging_fraction=0.5262728428461787, bagging_freq=3,
              early_stopping_round=100, feature_fraction=0.5,
              feature_pre_filter=False, lambda_l1=3.731605225849285,
              lambda_l2=0.02803980626777797, metric='l1', n_jobs=4,
              num_iterations=10000, num_leaves=8, objective='mae', seed=42)

In [58]:

#
model3 = lgb.LGBMRegressor(**params1)
model3.fit(dtr_g3.drop(columns=col_label+col_key).values,dtr_g3['target3'].values,
           eval_set=[(dtt_g3.drop(columns=col_label+col_key).values,dtt_g3['target3'].values)],verbose=100)


Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 1.71576
Early stopping, best iteration is:
[8]	valid_0's l1: 1.65176


LGBMRegressor(bagging_fraction=0.9102311988250655, bagging_freq=7,
              early_stopping_round=100, feature_fraction=0.6577763017247104,
              learning_rate=0.06400959575202655, max_bin=166, max_depth=11,
              min_sum_hessian_in_leaf=8, n_estimators=4492, n_jobs=4,
              num_iterations=10000, num_leaves=36693, objective='mae',
              reg_alpha=0.00022893773473705515,
              reg_lambda=0.00048731841698770753)

In [59]:

#
model4 = lgb.LGBMRegressor(**params1)
model4.fit(dtr_g4.drop(columns=col_label+col_key).values,dtr_g4['target4'].values,
           eval_set=[(dtt_g4.drop(columns=col_label+col_key).values,dtt_g4['target4'].values)],verbose=100)


Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 1.32037
[200]	valid_0's l1: 1.33024
Early stopping, best iteration is:
[108]	valid_0's l1: 1.31478


LGBMRegressor(bagging_fraction=0.9102311988250655, bagging_freq=7,
              early_stopping_round=100, feature_fraction=0.6577763017247104,
              learning_rate=0.06400959575202655, max_bin=166, max_depth=11,
              min_sum_hessian_in_leaf=8, n_estimators=4492, n_jobs=4,
              num_iterations=10000, num_leaves=36693, objective='mae',
              reg_alpha=0.00022893773473705515,
              reg_lambda=0.00048731841698770753)

In [41]:

if SAVE:
    joblib.dump(model1,DATA_PATH/Path('t1g_'+lag_name+'.pkl'))
    joblib.dump(model2,DATA_PATH/Path('t2g_'+lag_name+'.pkl'))
    joblib.dump(model3,DATA_PATH/Path('t3g_'+lag_name+'.pkl'))
    joblib.dump(model4,DATA_PATH/Path('t4g_'+lag_name+'.pkl'))

In [60]:
pred1 = model1.predict(dtt_g1.drop(columns=col_label+col_key).values)
pred2 = model2.predict(dtt_g2.drop(columns=col_label+col_key).values)
pred3 = model3.predict(dtt_g3.drop(columns=col_label+col_key).values)
pred4 = model4.predict(dtt_g4.drop(columns=col_label+col_key).values)

score_lgb1 = mean_absolute_error(pred1, dtt_g1['target1'].values)
score_lgb2 = mean_absolute_error(pred2, dtt_g2['target2'].values)
score_lgb3 = mean_absolute_error(pred3, dtt_g3['target3'].values)
score_lgb4 = mean_absolute_error(pred4, dtt_g4['target4'].values)
score1 = (score_lgb1+score_lgb2+score_lgb3+score_lgb4)/4
print('mae:', score_lgb1,score_lgb2,score_lgb3,score_lgb4)
print('score1:', score1)

mae: 2.492276253801739 3.6182724875438037 1.6517622877149556 1.3147831460768462
score1: 2.2692735437843363


## no game

In [61]:
# nogame

model1 = lgb.LGBMRegressor(**params1)
model1.fit(dtr_n1.drop(columns=col_label+col_key).values,dtr_n1['target1'].values,
           eval_set=[(dtt_n1.drop(columns=col_label+col_key).values,dtt_n1['target1'].values)],verbose=500)

model2 = lgb.LGBMRegressor(**params1)
model2.fit(dtr_n2.drop(columns=col_label+col_key).values,dtr_n2['target2'].values,
           eval_set=[(dtt_n2.drop(columns=col_label+col_key).values,dtt_n2['target2'].values)],verbose=500)


model3 = lgb.LGBMRegressor(**params1)
model3.fit(dtr_n3.drop(columns=col_label+col_key).values,dtr_n3['target3'].values,
           eval_set=[(dtt_n3.drop(columns=col_label+col_key).values,dtt_n3['target3'].values)],verbose=500)

model4 = lgb.LGBMRegressor(**params1)
model4.fit(dtr_n4.drop(columns=col_label+col_key).values,dtr_n4['target4'].values,
           eval_set=[(dtt_n4.drop(columns=col_label+col_key).values,dtt_n4['target4'].values)],verbose=500)


[LightGBM] [Warning] bagging_fraction is set=0.9102311988250655, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9102311988250655
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Warning] feature_fraction is set=0.6577763017247104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6577763017247104
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[14]	valid_0's l1: 0.640637
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's l1: 0.973838
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10]	valid_0's l1: 0.562885
Training until validation scores don't improve for 100 rounds
Early stopping, bes

LGBMRegressor(bagging_fraction=0.9102311988250655, bagging_freq=7,
              early_stopping_round=100, feature_fraction=0.6577763017247104,
              learning_rate=0.06400959575202655, max_bin=166, max_depth=11,
              min_sum_hessian_in_leaf=8, n_estimators=4492, n_jobs=4,
              num_iterations=10000, num_leaves=36693, objective='mae',
              reg_alpha=0.00022893773473705515,
              reg_lambda=0.00048731841698770753)

In [44]:
if SAVE:
    joblib.dump(model4,DATA_PATH/Path('t4n_'+lag_name+'.pkl'))
    joblib.dump(model3,DATA_PATH/Path('t3n_'+lag_name+'.pkl'))
    joblib.dump(model2,DATA_PATH/Path('t2n_'+lag_name+'.pkl'))
    joblib.dump(model1,DATA_PATH/Path('t1n_'+lag_name+'.pkl'))

In [62]:
pred1 = model1.predict(dtt_n1.drop(columns=col_label+col_key).values)
pred2 = model2.predict(dtt_n2.drop(columns=col_label+col_key).values)
pred3 = model3.predict(dtt_n3.drop(columns=col_label+col_key).values)
pred4 = model4.predict(dtt_n4.drop(columns=col_label+col_key).values)

score_lgb1 = mean_absolute_error(pred1, dtt_n1['target1'].values)
score_lgb2 = mean_absolute_error(pred2, dtt_n2['target2'].values)
score_lgb3 = mean_absolute_error(pred3, dtt_n3['target3'].values)
score_lgb4 = mean_absolute_error(pred4, dtt_n4['target4'].values)
score2 = (score_lgb1+score_lgb2+score_lgb3+score_lgb4)/4
print('mae:', score_lgb1,score_lgb2,score_lgb3,score_lgb4)
print('score2:', score2)

mae: 0.6406371877049902 0.973838018817104 0.5628852319313777 0.4887799371271491
score2: 0.6665350938951553


In [63]:
(score1+score2)/2

1.4679043188397458